## Основные задачи для работы с таблицами и Excel
### (выполнил Валяев Георгий Анатольевич, студент группы ПИ20-6)

Датасет: себестоимостьА_в1.xlsx

In [2]:
!pip install xlwings

     ---------------------------------------- 1.5/1.5 MB 832.9 kB/s eta 0:00:00


You should consider upgrading via the 'C:\Users\admin\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\admin\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [1]:
import xlwings as xw
import numpy as np
import pandas as pd

### 1. Используя xlwings:

In [2]:
wb = xw.Book('себестоимостьА_в1_после.xlsx')
sheet = wb.sheets['Рецептура']
sheet.range("C7:C10").value

['Хлеб Венгерский', 'Хлеб "Барбари"', 'Багет парижский', 'Чиабатта']

1.1 На листе "Рецептура" для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.


In [3]:
consumption = sheet.range("G7:O10").options(np.array).value
unit_price = sheet.range("G14:O14").options(np.array).value
consumption

array([[0.2  , 0.4  , 0.02 , 0.02 , 0.01 , 0.005, 0.1  ,   nan,   nan],
       [0.07 , 0.25 , 0.01 , 0.06 , 0.015,   nan, 0.06 , 0.05 ,   nan],
       [0.12 , 0.22 , 0.005,   nan, 0.009, 0.005,   nan, 0.05 , 0.03 ],
       [0.12 , 0.23 , 0.01 ,   nan, 0.015,   nan,   nan,   nan, 0.05 ]])

In [4]:
unit_price

array([  7.,  10., 184.,  19.,   7., 290., 105.,  83., 260.])

In [5]:
cost = np.nan_to_num(consumption * unit_price).sum(axis=1)
cost

array([21.48 , 16.525, 17.423, 18.085])

1.2 Результаты рассчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

In [12]:
sheet.range("T7").options(transpose=True).value = cost

1.3 Максимально приблизить форматирование столбца, добавленного в 1.2 к оформлению всей области.

In [11]:
xw.Range("T7").expand().number_format = '0,00'
xw.Range("T7:T10").color = (255, 255, 167)
sheet.range("T6").value = 'Себестоимость'
sheet.range("T4:T6").merge()
xw.Range("T4:T6").color = (255, 192, 0)
xw.Range("T4:T6").autofit()

#### Автоматизация пунктов 1.2 и 1.3 в функции routine_man()

In [6]:
def routine_man(consum, price, column):
    # column = "T 6"
    lst_data, lst_addresses = column.split(), []
    lst_addresses.append("".join(lst_data))
    cost_str = lst_data[0] + str(int(lst_data[1]) + 1)
    lst_addresses.append(cost_str)
    merge_str = lst_data[0] + str(int(lst_data[1]) - 2) + ":" + lst_addresses[0]
    lst_addresses.append(merge_str)

    # cost_str = "T" + str(6 + 1) - T7
    # merge_str = "T" + str(6 - 2) + ":" + "T6" - T4:T6
    # lst_addresses = ["T6", "T7", "T4:T6"]

    # работа с данными
    consumption = sheet.range(consum).options(np.array).value
    unit_price = sheet.range(price).options(np.array).value
    cost = np.nan_to_num(consumption * unit_price).sum(axis=1)

    # Создание столбца Себестоимости
    sheet.range(lst_addresses[0]).value = 'Себестоимость'
    sheet.range(lst_addresses[1]).options(transpose=True).value = cost

    # Заполнение данными ячеек по строкам + преобразование цвета и формата
    xw.Range(lst_addresses[1]).expand().number_format = '0,00'
    xw.Range(lst_addresses[1]).expand().color = (255, 255, 167)
    sheet.range(lst_addresses[2]).merge()
    xw.Range(lst_addresses[2]).color = (255, 192, 0)
    xw.Range(lst_addresses[2]).autofit()

    return cost

In [7]:
routine_man("G7:O10", "G14:O14", "T 6") # — вернёт аналогичный результат

array([21.48 , 16.525, 17.423, 18.085])

1.4 Заменить значения добавленные в 1.2 на формулы.

In [8]:
xw.Range("U6").value = 'Формулы'
sheet.range("U4:U6").merge()
xw.Range("U4:U6").color = (255, 192, 0)
xw.Range("U4:U6").autofit()
for i in [7, 8, 9, 10]:
    xw.Range(f"U{i}").formula = f'=SUMPRODUCT(G{i}:O{i},$G$14:$O$14)'
xw.Range("U7:U10").number_format = '0,00'
xw.Range("U7:U10").color = (255, 255, 167)

1.5 Выполнить 1.4 с помощью "протягиваемых" формул.

In [9]:
xw.Range("V6").value = 'Протяг формулы'
sheet.range("V4:V6").merge()
xw.Range("V4:V6").color = (255, 192, 0)
xw.Range("V4:V6").autofit()

fml = xw.Range("V7").formula = '=SUMPRODUCT(G7:O7,$G$14:$O$14)'
xw.Range("V7:V10").formula = fml
xw.Range("V7").number_format = '0,00'
xw.Range("V7").color = (255, 255, 167)

#### Автоматизация пункта 1.5 в функции stretch_formula_man()

In [13]:
def stretch_formula_man(data_size, prices, column):

    # column = "V 6"
    # data_size = "G7:O10"
    # prices = "G14:O14"

    lst_data, lst_addresses = column.split(), []
    lst_addresses.append("".join(lst_data))
    formula_str = lst_data[0] + str(int(lst_data[1]) + 1)
    lst_addresses.append(formula_str)
    merge_str = lst_data[0] + str(int(lst_data[1]) - 2) + ":" + lst_addresses[0]
    lst_addresses.append(merge_str)
    consumption = sheet.range(data_size).options(np.array).value
    unit_price = sheet.range(prices).options(np.array).value
    formula_len_str = len(np.nan_to_num(consumption * unit_price).sum(axis=1))
    final_len_str = lst_addresses[1] + ":" + lst_data[0] + str(int(lst_data[1]) + formula_len_str)
    lst_addresses.append(final_len_str)

    borders_data = data_size.split(":") # ["G7", "O10"]
    down_symbol_data = "".join([elem if not elem.isdigit() else "" for elem in borders_data[1]]) + str(int(lst_data[1]) + 1) # "O7"
    one_part_f = borders_data[0] + ":" + down_symbol_data
    lst_addresses.append(one_part_f)

    borders_price = prices.split(":") # ["G14", "O14"]
    one_s_part = "$" + "".join([elem if not elem.isdigit() else "" for elem in borders_price[0]]) + "$" + "".join([elem if elem.isdigit() else "" for elem in borders_price[0]]) # "$G$14"
    second_s_part = "$" + "".join([elem if not elem.isdigit() else "" for elem in borders_price[1]]) + "$" + "".join([elem if elem.isdigit() else "" for elem in borders_price[1]]) # "$O$14"
    second_part_f = one_s_part + ":" + second_s_part
    lst_addresses.append(second_part_f)

    # formula_str = "V" + str(6 + 1) - V7
    # merge_str = "V" + str(6 - 2) + ":" + "V6" - V4:V6
    # formula_len_str = длина выводимых данных через прямой расчёт — 4
    # final_len_str = "V7" + ":" + "V" + str(6 + 4) — "V7:V10"
    # one_part_f = "G7" + ":" + "O7" — "G7:O7"
    # second_part_f = "$G$14" + ":" + "$O$14" — "$G$14:$O$14"
    # lst_addresses = ["V6", "V7", "V4:V6, "V7:V10", "G7:O7", "$G$14:$O$14"]

    # Наименование столбца и работа над его форматом
    xw.Range(lst_addresses[0]).value = 'Stretch формулы'
    xw.Range(lst_addresses[2]).merge()
    xw.Range(lst_addresses[2]).color = (255, 192, 0)
    xw.Range(lst_addresses[2]).autofit()

    # Взаимодействие с данными и формулой
    fml = xw.Range(lst_addresses[1]).formula = f'=SUMPRODUCT({lst_addresses[4]},{lst_addresses[5]})'
    xw.Range(lst_addresses[3]).formula = fml
    xw.Range(lst_addresses[3]).number_format = '0,00'
    xw.Range(lst_addresses[3]).color = (255, 255, 167)

In [17]:
stretch_formula_man("G7:O10", "G14:O14", "V 6") # — вернёт аналогичный результат

1.6 Сделать так, чтобы решение 1.5 и 1.3 автоматически применилось ко всем аналогичным областям на листе "Рецептура"

##### Область «Ржаной хлеб»

In [11]:
# Решение 1.3
routine_man("G23:N25", "G31:N31", "S 22")

array([14.345, 29.75 , 10.138])

In [12]:
# Решение 1.5
stretch_formula_man("G23:N25", "G31:N31", "T 22")

##### Область «Хлеба специальной культуры»

In [18]:
# Решение 1.3
routine_man("G40:X46", "G52:X52", "AC 39")

array([38.575, 19.42 , 37.055, 29.54 , 28.33 , 22.615, 32.14 ])

In [19]:
# Решение 1.5
stretch_formula_man("G40:X46", "G52:X52", "AD 39")

##### Область «Слоеная выпечка»

In [20]:
# Решение 1.3
routine_man("G61:Y69", "G73:Y73", "AD 60")

array([ 7.055     ,  9.415     ,  9.055     ,  7.37      , 11.172171  ,
       17.38020635, 10.49147619,  9.27419048, 26.58842857])

In [21]:
# Решение 1.5
stretch_formula_man("G61:Y69", "G73:Y73", "AE 60")

### 2. Используя xlwings:

2.1 Собрать цены всех ресурсов, используемые на листе "Рецептура". Проверить, что нет расхождений между ценами совпадающих ресурсов

In [22]:
def collect_all_res(lst_tuples_new_res):
    all_resources = xw.Range("G5:O14").value[0]
    all_prices = xw.Range("G5:O14").value[-1]
    final_set = list(zip(all_resources, all_prices))

    for address in lst_tuples_new_res:
        final_set.extend(list(zip(xw.Range(f"{address[0]}:{address[1]}").value[0], xw.Range(f"{address[0]}:{address[1]}").value[-1])))

    final_set = dict(set(final_set))
    final_set = {key: final_set[key] for key in sorted(list(final_set.keys()))}
    return final_set

all_resources = collect_all_res([("G21", "N31"), ("G38", "X52"), ("G59", "Y73")])
print(all_resources)

{'Ветчина': 270.0, 'Вишня': 93.0, 'Вода': 7.0, 'Гречневая мука': 12.0, 'Джем': 63.0, 'Дрожжи': 184.0, 'Зелень': 280.0, 'Изюм': 83.0, 'Кукурузная мука': 12.0, 'Кунжутные семена': 83.0, 'Куркума молотая': 290.0, 'Лосось': 250.0, 'Масло оливковое': 260.0, 'Масло сливочное': 150.0, 'Молоко': 40.0, 'Овсянные хлопья': 18.0, 'Пахта': 40.0, 'Пшеничная мука': 10.0, 'Ржаная мука': 10.0, 'Сахар': 19.0, 'Семена подсолнуха': 160.0, 'Солод ржаной': 108.0, 'Соль': 7.0, 'Сыр': 110.0, 'Творог': 80.0, 'Топленное масло': 105.0, 'Цельнозерновая мука': 12.0, 'Чеснок ': 1.0, 'Шоколад': 200.0, 'Яблоки': 25.0, 'Яйцо': 4.0, 'Ячменная мука': 12.0}


2.2 Вынести цены ресурсов на новый лист "Цена ресурсов".

In [23]:
sheet_2 = wb.sheets.add(name="Цена ресурсов", after=wb.sheets[0].name)
sheet_2.range("A1").value = 'Фиксированные цены'
sheet_2.range("A1:B1").merge()
xw.Range("A1").expand('right').color = (255, 192, 0)
sheet_2.range("A2").value = ["Ресурс", "Цена"]
xw.Range("A2").expand().color = (255, 192, 0)
sheet_2.range("A3").value = all_resources
xw.Range("B3").expand().color = (255, 255, 167)
sheet_2.range("A2").expand().autofit()

2.3 Заменить фиксированные цены ресурсов на листе "Рецептура" на ссылки на лист "Цена ресурсов"

In [24]:
wb.sheets[0].activate()
resources = list(all_resources.keys())
addr_prices = [f'B{i}' for i in range(3, 35)]
dict_res_pr = dict(zip(resources, addr_prices))
dict_res_pr

{'Ветчина': 'B3',
 'Вишня': 'B4',
 'Вода': 'B5',
 'Гречневая мука': 'B6',
 'Джем': 'B7',
 'Дрожжи': 'B8',
 'Зелень': 'B9',
 'Изюм': 'B10',
 'Кукурузная мука': 'B11',
 'Кунжутные семена': 'B12',
 'Куркума молотая': 'B13',
 'Лосось': 'B14',
 'Масло оливковое': 'B15',
 'Масло сливочное': 'B16',
 'Молоко': 'B17',
 'Овсянные хлопья': 'B18',
 'Пахта': 'B19',
 'Пшеничная мука': 'B20',
 'Ржаная мука': 'B21',
 'Сахар': 'B22',
 'Семена подсолнуха': 'B23',
 'Солод ржаной': 'B24',
 'Соль': 'B25',
 'Сыр': 'B26',
 'Творог': 'B27',
 'Топленное масло': 'B28',
 'Цельнозерновая мука': 'B29',
 'Чеснок ': 'B30',
 'Шоколад': 'B31',
 'Яблоки': 'B32',
 'Яйцо': 'B33',
 'Ячменная мука': 'B34'}

In [25]:
for pair in [("G5", "G14"), ("G21", "G31"), ("G38", "G52"), ("G59", "G73")]:
    list_links = [f"='Цена ресурсов'!{dict_res_pr[resource]}" for resource in sheet.range(pair[0]).expand().value]
    sheet.range(pair[1]).expand('right').value = list_links
    print()
    print(*list_links, sep='\n')
    print()


='Цена ресурсов'!B5
='Цена ресурсов'!B20
='Цена ресурсов'!B8
='Цена ресурсов'!B22
='Цена ресурсов'!B25
='Цена ресурсов'!B13
='Цена ресурсов'!B28
='Цена ресурсов'!B12
='Цена ресурсов'!B15


='Цена ресурсов'!B5
='Цена ресурсов'!B21
='Цена ресурсов'!B20
='Цена ресурсов'!B8
='Цена ресурсов'!B22
='Цена ресурсов'!B25
='Цена ресурсов'!B15
='Цена ресурсов'!B24


='Цена ресурсов'!B5
='Цена ресурсов'!B29
='Цена ресурсов'!B6
='Цена ресурсов'!B34
='Цена ресурсов'!B11
='Цена ресурсов'!B20
='Цена ресурсов'!B8
='Цена ресурсов'!B22
='Цена ресурсов'!B25
='Цена ресурсов'!B13
='Цена ресурсов'!B17
='Цена ресурсов'!B12
='Цена ресурсов'!B15
='Цена ресурсов'!B30
='Цена ресурсов'!B18
='Цена ресурсов'!B19
='Цена ресурсов'!B10
='Цена ресурсов'!B23


='Цена ресурсов'!B5
='Цена ресурсов'!B20
='Цена ресурсов'!B8
='Цена ресурсов'!B22
='Цена ресурсов'!B25
='Цена ресурсов'!B17
='Цена ресурсов'!B16
='Цена ресурсов'!B33
='Цена ресурсов'!B15
='Цена ресурсов'!B7
='Цена ресурсов'!B26
='Цена ресурсов'!B3
='Цена ресурсов'!

2.4 Сделать так, чтобы цены ресурсов на листе "Рецептура" подтягивались с листа "Цена ресурсов" по наименованию (подсказка: использовать ВПР, рекомендуется использовать именованные области)

In [26]:
resources = list(all_resources.keys())
addr_prices = [f'A{i}' for i in range(3, 35)]
dict_res_pr = dict(zip(resources, addr_prices))
dict_res_pr

{'Ветчина': 'A3',
 'Вишня': 'A4',
 'Вода': 'A5',
 'Гречневая мука': 'A6',
 'Джем': 'A7',
 'Дрожжи': 'A8',
 'Зелень': 'A9',
 'Изюм': 'A10',
 'Кукурузная мука': 'A11',
 'Кунжутные семена': 'A12',
 'Куркума молотая': 'A13',
 'Лосось': 'A14',
 'Масло оливковое': 'A15',
 'Масло сливочное': 'A16',
 'Молоко': 'A17',
 'Овсянные хлопья': 'A18',
 'Пахта': 'A19',
 'Пшеничная мука': 'A20',
 'Ржаная мука': 'A21',
 'Сахар': 'A22',
 'Семена подсолнуха': 'A23',
 'Солод ржаной': 'A24',
 'Соль': 'A25',
 'Сыр': 'A26',
 'Творог': 'A27',
 'Топленное масло': 'A28',
 'Цельнозерновая мука': 'A29',
 'Чеснок ': 'A30',
 'Шоколад': 'A31',
 'Яблоки': 'A32',
 'Яйцо': 'A33',
 'Ячменная мука': 'A34'}

In [27]:
for pair in [("G5", "G14"), ("G21", "G31"), ("G38", "G52"), ("G59", "G73")]:
    list_links = [f"=VLOOKUP('Цена ресурсов'!{dict_res_pr[resource]},Цены_ресурсы,2,0)" for resource in sheet.range(pair[0]).expand().value]
    sheet.range(pair[1]).expand('right').value = list_links
    print()
    print(*list_links, sep='\n')
    print()


=VLOOKUP('Цена ресурсов'!A5,Цены_ресурсы,2,0)
=VLOOKUP('Цена ресурсов'!A20,Цены_ресурсы,2,0)
=VLOOKUP('Цена ресурсов'!A8,Цены_ресурсы,2,0)
=VLOOKUP('Цена ресурсов'!A22,Цены_ресурсы,2,0)
=VLOOKUP('Цена ресурсов'!A25,Цены_ресурсы,2,0)
=VLOOKUP('Цена ресурсов'!A13,Цены_ресурсы,2,0)
=VLOOKUP('Цена ресурсов'!A28,Цены_ресурсы,2,0)
=VLOOKUP('Цена ресурсов'!A12,Цены_ресурсы,2,0)
=VLOOKUP('Цена ресурсов'!A15,Цены_ресурсы,2,0)


=VLOOKUP('Цена ресурсов'!A5,Цены_ресурсы,2,0)
=VLOOKUP('Цена ресурсов'!A21,Цены_ресурсы,2,0)
=VLOOKUP('Цена ресурсов'!A20,Цены_ресурсы,2,0)
=VLOOKUP('Цена ресурсов'!A8,Цены_ресурсы,2,0)
=VLOOKUP('Цена ресурсов'!A22,Цены_ресурсы,2,0)
=VLOOKUP('Цена ресурсов'!A25,Цены_ресурсы,2,0)
=VLOOKUP('Цена ресурсов'!A15,Цены_ресурсы,2,0)
=VLOOKUP('Цена ресурсов'!A24,Цены_ресурсы,2,0)


=VLOOKUP('Цена ресурсов'!A5,Цены_ресурсы,2,0)
=VLOOKUP('Цена ресурсов'!A29,Цены_ресурсы,2,0)
=VLOOKUP('Цена ресурсов'!A6,Цены_ресурсы,2,0)
=VLOOKUP('Цена ресурсов'!A34,Цены_ресурсы,2,0)
=VLOOKUP('Цена

### 3. Используя xlwings:

3.1 Создать нормализованную (не ниже 3й нормальной формы) базу данных sqlite для хранения информации из себестоимостьА_в1.xlsx .

3.2 Информацию из себестоимостьА_в1.xlsx внести в БД на sqlite.

3.3 По базе данных sqlite сгенерирвать книгу эксель, по структуре аналогичную себестоимостьА_в1.xlsx

3.4 При помощи макроса на xlwings сделать в Excel кнопку, по которой все изменения на листе вносятся в БД, а цвет ячеек, в которых были найдены изменения, меняслся на зеленый.

3.5 Реализовать UDF, которая позволяет по имени ресурса вернуть количество видов в продукции в которых используется данный ресурс

3.6 Реализовать UDF, которая позволяет по имени ресурса вернуть массив наименований продуктов, в которых используется данный ресурс

3.7 Реазливать UDF 3.5 и 3.6 которые для получения ответа использую не книгу Excel а БД на sqlite.